In [1]:
from pathlib import Path
import tomllib
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from blizzapi import Character, BlizzardAPIURLs, CharacterData

In [2]:
client_filepath = Path(r"D:\Programming\github-repos\nop-raid-bot\clientdetails.toml")
with open(client_filepath, "rb") as client_file:
    client_data = tomllib.load(client_file)

CLIENT_ID = client_data["client"]["id"]
CLIENT_SECRET = client_data["client"]["secret"]

TOKEN_URL = "https://oauth.battle.net/token"

In [3]:
client = BackendApplicationClient(client_id=CLIENT_ID)
oauth = OAuth2Session(client=client, state="blah123")
token = oauth.fetch_token(token_url=TOKEN_URL, client_id=CLIENT_ID, client_secret=CLIENT_SECRET)

{'access_token': 'EU0dR3G5sspjE6Ohggj14w1msA0yD5cmUy',
 'token_type': 'bearer',
 'expires_in': 86399,
 'sub': '1492badc8bfe49469af56f9adf787596',
 'expires_at': 1730133782.6484876}

In [4]:
testchar = Character("Aptosaurinae", "Draenor")
blizz_urls = BlizzardAPIURLs()
chardata = CharacterData(testchar, blizz_api_urls=blizz_urls, oauth=oauth)
chardata.get_specific_raid_data("The War Within", "Nerub-ar Palace", "Heroic")

"Nerub-ar Palace [Heroic]:\n- Progress: 3 / 8\n- Sikran, Captain of the Sureki: <t:1729536136:D>\n- Rasha'nan: <t:1729538172:D>\n- Broodtwister Ovi'nax: <t:1729542901:D>"

In [5]:
chardata.get_current_enchants(True)

"Enchants:\n- 8 / 8\n- Off Hand missing enchant\n- Back: Enchanted: +355 Avoidance :quality3:\n- Chest: Enchanted: Crystalline Radiance :quality2:\n- Wrist: Enchanted: +710 Avoidance :quality3:\n- Legs: Enchanted: +747 Intellect & +230 Stamina :quality3:\n- Feet: Enchanted: Scout's March :quality2:\n- Ring 1: Enchanted: +190 Haste :quality3:\n- Ring 2: Enchanted: +190 Haste :quality3:\n- Main Hand: Enchanted: Stormrider's Fury :quality3:"

In [6]:
equipment = oauth.get(blizz_urls.get_equipment(testchar)).json()
equipment

{'_links': {'self': {'href': 'https://eu.api.blizzard.com/profile/wow/character/draenor/aptosaurinae/equipment?namespace=profile-eu'}},
 'character': {'key': {'href': 'https://eu.api.blizzard.com/profile/wow/character/draenor/aptosaurinae?namespace=profile-eu'},
  'name': 'Aptosaurinae',
  'id': 177892318,
  'realm': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/realm/506?namespace=dynamic-eu'},
   'name': 'Draenor',
   'id': 506,
   'slug': 'draenor'}},
 'equipped_items': [{'item': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/item/212092?namespace=static-11.0.5_56865-eu'},
    'id': 212092},
   'sockets': [{'socket_type': {'type': 'PRISMATIC',
      'name': 'Prismatic Socket'},
     'item': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/item/213746?namespace=static-11.0.5_56865-eu'},
      'name': 'Elusive Blasphemite',
      'id': 213746},
     'display_string': '+181 Primary Stat and +2% Movement Speed per unique Algari gem color',
     'media': {'key': {'

In [7]:
[item["slot"]["name"] for item in equipment["equipped_items"]]

['Head',
 'Neck',
 'Shoulders',
 'Shirt',
 'Chest',
 'Waist',
 'Legs',
 'Feet',
 'Wrist',
 'Hands',
 'Ring 1',
 'Ring 2',
 'Trinket 1',
 'Trinket 2',
 'Back',
 'Main Hand',
 'Tabard']

In [11]:
encounters_url = blizz_urls.get_encounter_journal_index()
nerubar_url = blizz_urls.get_encounter_list(1273)
encounters_json = oauth.get(encounters_url).json()
nerubar_json = oauth.get(nerubar_url).json()
display(encounters_json, nerubar_json)

{'_links': {'self': {'href': 'https://eu.api.blizzard.com/data/wow/journal-instance/?namespace=static-11.0.5_56865-eu'}},
 'instances': [{'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-instance/64?namespace=static-11.0.5_56865-eu'},
   'name': 'Shadowfang Keep',
   'id': 64},
  {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-instance/67?namespace=static-11.0.5_56865-eu'},
   'name': 'The Stonecore',
   'id': 67},
  {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-instance/76?namespace=static-11.0.5_56865-eu'},
   'name': "Zul'Gurub",
   'id': 76},
  {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-instance/66?namespace=static-11.0.5_56865-eu'},
   'name': 'Blackrock Caverns',
   'id': 66},
  {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-instance/63?namespace=static-11.0.5_56865-eu'},
   'name': 'Deadmines',
   'id': 63},
  {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-instance/69?namespace=stati

{'_links': {'self': {'href': 'https://eu.api.blizzard.com/data/wow/journal-instance/1273?namespace=static-11.0.5_56865-eu'}},
 'id': 1273,
 'name': 'Nerub-ar Palace',
 'map': {'name': 'Nerub-ar Palace', 'id': 2657},
 'description': "Deep within the kingdom of Azj-Kahet lies Nerub-ar Palace, Queen Ansurek's seat of power. From this shadowed throne Queen Ansurek has tightened her grip on her Sureki followers, leading them along a dark path to a new future. As her paranoia grows, the members of the conspiratorial Severed Threads launch their final gambit against Ansurek's rule.",
 'encounters': [{'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-encounter/2607?namespace=static-11.0.5_56865-eu'},
   'name': 'Ulgrax the Devourer',
   'id': 2607},
  {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-encounter/2611?namespace=static-11.0.5_56865-eu'},
   'name': 'The Bloodbound Horror',
   'id': 2611},
  {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-enc

In [12]:
raid_query = blizz_urls.get_raids(testchar)
raid_query

'https://eu.api.blizzard.com/profile/wow/character/draenor/aptosaurinae/encounters/raids?locale=en_GB&namespace=profile-eu'

In [13]:
response = oauth.get(raid_query)
response

<Response [200]>

In [14]:
response.headers

{'Date': 'Sun, 27 Oct 2024 16:46:49 GMT', 'Content-Type': 'application/json;charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Battlenet-Schema-Revision': '2', 'Last-Modified': 'Sun, 27 Oct 2024 16:00:17 GMT', 'Battlenet-Namespace': 'profile-eu', 'vary': 'accept-encoding', 'blizzard-token-expires': '2024-10-28T16:43:04.736Z', 'Battlenet-Schema': 'character-raid-progression-raids', 'content-encoding': 'gzip', 'x-trace-traceid': '6c48996b-e9fd-3c1b-8c70-b44c0c050c88', 'x-trace-spanid': '1801b917-0eeb-82d0-f7e3-fa165bb18be7', 'x-trace-parentspanid': '1801b917-0eeb-82a0-f7e3-fa165bb18be7', 'x-frame-options': 'SAMEORIGIN', 'X-Content-Type-Options': 'nosniff', 'server': 'blizzard'}

In [15]:
response.json()

{'_links': {'self': {'href': 'https://eu.api.blizzard.com/profile/wow/character/draenor/aptosaurinae/encounters/raids?namespace=profile-eu'}},
 'character': {'key': {'href': 'https://eu.api.blizzard.com/profile/wow/character/draenor/aptosaurinae?namespace=profile-eu'},
  'name': 'Aptosaurinae',
  'id': 177892318,
  'realm': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/realm/506?namespace=dynamic-eu'},
   'name': 'Draenor',
   'id': 506,
   'slug': 'draenor'}},
 'expansions': [{'expansion': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-expansion/73?namespace=static-11.0.5_56865-eu'},
    'name': 'Cataclysm',
    'id': 73},
   'instances': [{'instance': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/journal-instance/72?namespace=static-11.0.5_56865-eu'},
      'name': 'The Bastion of Twilight',
      'id': 72},
     'modes': [{'difficulty': {'type': 'LEGACY_25_MAN_HEROIC',
        'name': '25 Player (Heroic)'},
       'status': {'type': 'COMPLETE', 'nam

In [ ]:
response.json()["expansions"]

In [ ]:
[data["expansion"]["name"] for data in response.json()["expansions"]]

In [17]:
expansions_list = [data["expansion"]["name"] for data in response.json()["expansions"]]
exp_idx = expansions_list.index("The War Within")
instance_names = [data["instance"]["name"] for data in response.json()["expansions"][exp_idx]["instances"]]
ins_idx = instance_names.index("Nerub-ar Palace")
difficulities = [data["difficulty"]["name"] for data in response.json()["expansions"][exp_idx]["instances"][ins_idx]["modes"]]
difficulty_index = difficulities.index("Heroic")

In [ ]:
response.json()["expansions"][exp_idx]["instances"][ins_idx]["modes"][difficulty_index]["progress"]